In [1]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
import shutil
from tensorflow.keras import backend as K
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from focal_loss import BinaryFocalLoss
from tensorflow.keras.optimizers.legacy import Adam
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

Wykorzystany zbiór: https://zenodo.org/records/6378474

In [2]:
# glowny folder
base_dir = '/Users/milenabiernacka/Desktop/studia/DS/semestr2/PD-magisterka/Mushroom_dataset/cnn/'
# polaczenie glownego folderu i podfolderow z edible i poisonous

edible_dir = os.path.join(base_dir, 'Edible')
poisonous_dir = os.path.join(base_dir, 'Poisonous')

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Assuming base_dir is the path to the dataset directory
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=1,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 1399 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


Found 301 images belonging to 2 classes.


In [24]:
model = Sequential([
    Input(shape=(150, 150, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Use linear activation for Hinge loss
])


In [25]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])


In [16]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20
1399/1399 [==============================] - 54s 39ms/step - loss: 0.5225 - accuracy: 0.7405 - val_loss: 0.4246 - val_accuracy: 0.7465
Epoch 2/20
1399/1399 [==============================] - 52s 37ms/step - loss: 0.3967 - accuracy: 0.8127 - val_loss: 0.2942 - val_accuracy: 0.8507
Epoch 3/20
1399/1399 [==============================] - 53s 38ms/step - loss: 0.3062 - accuracy: 0.8628 - val_loss: 0.3521 - val_accuracy: 0.8472
Epoch 4/20
1399/1399 [==============================] - 54s 39ms/step - loss: 0.2569 - accuracy: 0.8863 - val_loss: 0.1986 - val_accuracy: 0.9062
Epoch 5/20
1399/1399 [==============================] - 55s 39ms/step - loss: 0.2254 - accuracy: 0.9049 - val_loss: 0.1896 - val_accuracy: 0.9097
Epoch 6/20
1399/1399 [==============================] - 55s 39ms/step - loss: 0.1908 - accuracy: 0.9149 - val_loss: 0.1824 - val_accuracy: 0.9410
Epoch 7/20
1399/1399 [==============================] - 55s 39ms/step - loss: 0.1641 - accuracy: 0.9328 - val_loss: 0.1170 -

In [17]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 1s 76ms/step - loss: 0.0897 - accuracy: 0.9549
Test Loss: 0.08970168977975845, Test Accuracy: 0.9548611044883728


In [123]:
from tensorflow.keras.losses import BinaryFocalCrossentropy

In [27]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss=BinaryFocalCrossentropy(), metrics=['accuracy'])


In [12]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20
1399/1399 [==============================] - 76s 54ms/step - loss: 0.1396 - accuracy: 0.7370 - val_loss: 0.1276 - val_accuracy: 0.7535
Epoch 2/20
1399/1399 [==============================] - 55s 39ms/step - loss: 0.1042 - accuracy: 0.8213 - val_loss: 0.0819 - val_accuracy: 0.8472
Epoch 3/20
1399/1399 [==============================] - 79s 56ms/step - loss: 0.0777 - accuracy: 0.8585 - val_loss: 0.0532 - val_accuracy: 0.8889
Epoch 4/20
1399/1399 [==============================] - 86s 61ms/step - loss: 0.0630 - accuracy: 0.8942 - val_loss: 0.0499 - val_accuracy: 0.9132
Epoch 5/20
1399/1399 [==============================] - 87s 62ms/step - loss: 0.0479 - accuracy: 0.9171 - val_loss: 0.0362 - val_accuracy: 0.9271
Epoch 6/20
1399/1399 [==============================] - 87s 62ms/step - loss: 0.0405 - accuracy: 0.9357 - val_loss: 0.0261 - val_accuracy: 0.9549
Epoch 7/20
1399/1399 [==============================] - 86s 62ms/step - loss: 0.0326 - accuracy: 0.9543 - val_loss: 0.0210 -

In [13]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 3s 282ms/step - loss: 0.0492 - accuracy: 0.9618
Test Loss: 0.049241337925195694, Test Accuracy: 0.9618055820465088


## Test CSL - dodanie niestandardowej macierzy kosztu


In [1]:
class_weights = {0: 1.0, 
                 1: 100.0} 

In [19]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    class_weight = class_weights
)

Epoch 1/20
1399/1399 [==============================] - 52s 37ms/step - loss: 11.7525 - accuracy: 0.5861 - val_loss: 2.3241 - val_accuracy: 0.6806
Epoch 2/20
1399/1399 [==============================] - 53s 38ms/step - loss: 2.7768 - accuracy: 0.6848 - val_loss: 4.2395 - val_accuracy: 0.2951
Epoch 3/20
1399/1399 [==============================] - 54s 39ms/step - loss: 2.4826 - accuracy: 0.6805 - val_loss: 0.9758 - val_accuracy: 0.7431
Epoch 4/20
1399/1399 [==============================] - 55s 39ms/step - loss: 4.7902 - accuracy: 0.6112 - val_loss: 1.5368 - val_accuracy: 0.5069
Epoch 5/20
1399/1399 [==============================] - 55s 39ms/step - loss: 2.3593 - accuracy: 0.6226 - val_loss: 0.5859 - val_accuracy: 0.8646
Epoch 6/20
1399/1399 [==============================] - 56s 40ms/step - loss: 1.8827 - accuracy: 0.6969 - val_loss: 0.3701 - val_accuracy: 0.9062
Epoch 7/20
1399/1399 [==============================] - 56s 40ms/step - loss: 2.9231 - accuracy: 0.6898 - val_loss: 1.0959 

In [13]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 1s 72ms/step - loss: 0.1223 - accuracy: 0.9514
Test Loss: 0.12229922413825989, Test Accuracy: 0.9513888955116272


# Wykorzystanie pretrenowanych sieci

### MobileNetV2

In [4]:
from tensorflow.keras.applications import MobileNetV2

In [5]:
img_height = img_width = 128

# Ładowanie MobileNetV2 bez górnej warstwy (top)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Zamrożenie wagi przetrenowanego modelu
base_model.trainable = False

In [6]:
# Dodanie warstw na wierzchu
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
# To jest tylko przykład, można tutaj dostosować liczbę neuronów i warstw
output = Dense(1, activation='sigmoid')(x)  # Przykładowa warstwa wyjściowa dla klasyfikacji binarnej

In [7]:
# Skompletowanie nowego modelu
model = Model(inputs=base_model.input, outputs=output)

In [8]:
# Kompilacja modelu z niestandardową funkcją straty i optymalizatorem
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [11]:
# Wypisanie struktury modelu
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 64, 64, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 64, 64, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 64, 64, 32)           0         ['bn_Conv1[0][0]']        

In [9]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/50
1399/1399 [==============================] - 21s 14ms/step - loss: 0.1476 - accuracy: 0.9428 - val_loss: 0.0394 - val_accuracy: 0.9861
Epoch 2/50
1399/1399 [==============================] - 23s 16ms/step - loss: 0.0825 - accuracy: 0.9671 - val_loss: 0.0445 - val_accuracy: 0.9861
Epoch 3/50
1399/1399 [==============================] - 26s 18ms/step - loss: 0.0401 - accuracy: 0.9850 - val_loss: 0.0200 - val_accuracy: 0.9931
Epoch 4/50
1399/1399 [==============================] - 29s 21ms/step - loss: 0.0264 - accuracy: 0.9907 - val_loss: 0.0141 - val_accuracy: 0.9965
Epoch 5/50
1399/1399 [==============================] - 28s 20ms/step - loss: 0.0489 - accuracy: 0.9864 - val_loss: 0.0193 - val_accuracy: 0.9965
Epoch 6/50
1399/1399 [==============================] - 27s 19ms/step - loss: 0.0361 - accuracy: 0.9864 - val_loss: 0.0296 - val_accuracy: 0.9826
Epoch 7/50
1399/1399 [==============================] - 28s 20ms/step - loss: 0.0070 - accuracy: 0.9979 - val_loss: 0.0422 -

In [11]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 1s 86ms/step - loss: 0.0060 - accuracy: 1.0000
Test Loss: 0.006028125062584877, Test Accuracy: 1.0


# Dane tabelaryczne 

Wykorzystany zbiór: https://mushroom.mathematik.uni-marburg.de

In [5]:
import pandas as pd

In [87]:
data_shuffled = pd.read_csv("/Users/milenabiernacka/Desktop/studia/magisterka/UCI/secondary_data_shuffled.csv", sep = ';')
data_no_miss = pd.read_csv('/Users/milenabiernacka/Desktop/studia/magisterka/UCI/secondary_data_no_miss.csv', sep = ';')

In [88]:
data_no_miss.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
0,e,1.26,x,g,y,f,d,c,w,5.04,1.73,y,f,f,d,a
1,e,10.32,f,e,b,f,a,c,b,4.68,19.44,w,t,f,d,a
2,p,0.92,x,g,p,f,a,c,p,4.59,1.15,k,f,f,d,u
3,p,4.27,x,t,p,f,x,c,w,4.55,6.52,w,f,f,d,a
4,e,3.08,f,s,w,f,d,d,w,2.67,5.18,w,f,f,m,a


In [89]:
data_no_miss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61069 entries, 0 to 61068
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   class                 61069 non-null  object 
 1   cap-diameter          61069 non-null  float64
 2   cap-shape             61069 non-null  object 
 3   cap-surface           61069 non-null  object 
 4   cap-color             61069 non-null  object 
 5   does-bruise-or-bleed  61069 non-null  object 
 6   gill-attachment       61069 non-null  object 
 7   gill-spacing          61069 non-null  object 
 8   gill-color            61069 non-null  object 
 9   stem-height           61069 non-null  float64
 10  stem-width            61069 non-null  float64
 11  stem-color            61069 non-null  object 
 12  has-ring              61069 non-null  object 
 13  ring-type             61069 non-null  object 
 14  habitat               61069 non-null  object 
 15  season             

In [90]:
y = data_no_miss['class']

In [91]:
y.head()

0    e
1    e
2    p
3    p
4    e
Name: class, dtype: object

In [92]:
X = data_no_miss.drop(columns='class')

In [93]:
X.head()

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
0,1.26,x,g,y,f,d,c,w,5.04,1.73,y,f,f,d,a
1,10.32,f,e,b,f,a,c,b,4.68,19.44,w,t,f,d,a
2,0.92,x,g,p,f,a,c,p,4.59,1.15,k,f,f,d,u
3,4.27,x,t,p,f,x,c,w,4.55,6.52,w,f,f,d,a
4,3.08,f,s,w,f,d,d,w,2.67,5.18,w,f,f,m,a


In [94]:
X = pd.get_dummies(X, dtype=int)
y = pd.get_dummies(y, dtype= int)

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [96]:
X_train

,cap-diameter,stem-height,stem-width,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_o,cap-shape_p,cap-shape_s,cap-shape_x,...,habitat_h,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w,season_a,season_s,season_u,season_w
42308,6.57,4.82,7.71,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
39798,5.13,5.38,14.26,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
40487,9.42,2.34,11.80,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
27727,3.11,6.57,6.05,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
25153,2.88,3.42,3.39,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54343,8.75,5.84,13.57,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
38158,9.17,9.32,27.01,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
860,5.55,4.80,13.36,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
15795,5.16,5.13,6.15,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [97]:
y_train

,e,p
42308,0,1
39798,1,0
40487,1,0
27727,0,1
25153,0,1
...,...,...
54343,1,0
38158,0,1
860,0,1
15795,0,1


In [98]:
X_train_encoded = pd.get_dummies(X_train, drop_first=True)

In [99]:
X_train_encoded

,cap-diameter,stem-height,stem-width,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_o,cap-shape_p,cap-shape_s,cap-shape_x,...,habitat_h,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w,season_a,season_s,season_u,season_w
42308,6.57,4.82,7.71,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
39798,5.13,5.38,14.26,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
40487,9.42,2.34,11.80,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
27727,3.11,6.57,6.05,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
25153,2.88,3.42,3.39,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54343,8.75,5.84,13.57,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
38158,9.17,9.32,27.01,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
860,5.55,4.80,13.36,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
15795,5.16,5.13,6.15,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [100]:
print(X_train_encoded.dtypes)

cap-diameter    float64
stem-height     float64
stem-width      float64
cap-shape_b       int64
cap-shape_c       int64
                 ...   
habitat_w         int64
season_a          int64
season_s          int64
season_u          int64
season_w          int64
Length: 92, dtype: object


In [101]:
y_train_encoded = pd.get_dummies(y_train, drop_first=True)

In [128]:
num_features = len(X_train.columns)

model = Sequential([
    # Warstwa wejściowa - określ, ile cech będzie przetwarzanych.
    Input(shape=(num_features,)),
    
    # Pierwsza warstwa ukryta
    Dense(128, activation='relu'),
    
    # Warstwa wyjściowa z jednym neuronem - używana dla predykcji binarnej
    Dense(2, activation='softmax')
])

# Kompilacja modelu
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [129]:
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
1527/1527 [==============================] - 7s 4ms/step - loss: 0.2501 - accuracy: 0.9030 - val_loss: 0.0695 - val_accuracy: 0.9885
Epoch 2/10
1527/1527 [==============================] - 6s 4ms/step - loss: 0.0337 - accuracy: 0.9960 - val_loss: 0.0159 - val_accuracy: 0.9991
Epoch 3/10
1527/1527 [==============================] - 6s 4ms/step - loss: 0.0102 - accuracy: 0.9994 - val_loss: 0.0058 - val_accuracy: 0.9998
Epoch 4/10
1527/1527 [==============================] - 6s 4ms/step - loss: 0.0043 - accuracy: 0.9998 - val_loss: 0.0026 - val_accuracy: 0.9998
Epoch 5/10
1527/1527 [==============================] - 6s 4ms/step - loss: 0.0032 - accuracy: 0.9997 - val_loss: 0.0019 - val_accuracy: 0.9999
Epoch 6/10
1527/1527 [==============================] - 6s 4ms/step - loss: 0.0014 - accuracy: 0.9999 - val_loss: 9.6721e-04 - val_accuracy: 0.9999
Epoch 7/10
1527/1527 [==============================] - 6s 4ms/step - loss: 0.0013 - accuracy: 0.9998 - val_loss: 5.3241e-04 - val_a

In [130]:
model.evaluate(X_test, y_test)

382/382 [==============================] - 1s 3ms/step - loss: 3.0197e-04 - accuracy: 1.0000


[0.0003019738069269806, 1.0]

In [131]:
predictions = model.predict(X_test)

382/382 [==============================] - 1s 2ms/step


In [132]:
y_pred = np.argmax(predictions, axis=1)
y_true = np.argmax(y_test, axis=1)

In [133]:
confusion_matrix(y_true, y_pred)

array([[5440,    0],
       [   0, 6774]])

In [134]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5440
           1       1.00      1.00      1.00      6774

    accuracy                           1.00     12214
   macro avg       1.00      1.00      1.00     12214
weighted avg       1.00      1.00      1.00     12214

